In [2]:
pip install atproto

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.1/337.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.7/671.7 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 14.1
    Uninstalling websockets-14.1:
      Successfully uninstalled websockets-14.1
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


In [3]:
# From the transformers package, import ViTFeatureExtractor and ViTForImageClassification
from transformers import ViTFeatureExtractor, ViTForImageClassification

# From the PIL package, import Image and Markdown
from PIL import Image

# import requests
import requests

# import torch
import torch

# import matplotlib
import matplotlib.pyplot as plt

# url getter for mpl
import urllib

import numpy as np

# import bluesky api
from atproto import Client

# import colab secrets to store login credentials
from google.colab import userdata

# datetime is necessary for caturday check
import datetime
import zoneinfo

In [4]:
# vision transformer documentation - https://huggingface.co/docs/transformers/v4.13.0/en/model_doc/vit
# guide to using vision transformer from DataCamp - https://www.youtube.com/watch?v=WQMDZniQgj4

# Load the feature extractor for the vision transformer
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
# Load the pre-trained weights from vision transformer
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [15]:
# 281: 'tabby, tabby cat'
# 282: 'tiger cat', 283: 'Persian cat', 284: 'Siamese cat, Siamese', 285: 'Egyptian cat', 286: 'cougar, puma, catamount, mountain lion, painter, panther, Felis concolor', 287: 'lynx, catamount', 288: 'leopard, Panthera pardus', 289: 'snow leopard, ounce, Panthera uncia', 290: 'jaguar, panther, Panthera onca, Felis onca', 291: 'lion, king of beasts, Panthera leo', 292: 'tiger, Panthera tigris', 293: 'cheetah, chetah, Acinonyx jubatus',
# 281 to 293
cat_labels = set()
for i in range(281, 294):
  cat_labels.add(i)

# these labels are to remove drawings, memes/reposts, and images with a lot of text respectively
bad_labels = {
917 : 'comic book', 916 : 'web site, website, internet site, site', 921 : 'book jacket, dust cover, dust jacket, dust wrapper'}

def test_bsky_image(url):

  f = urllib.request.urlopen(url)
  image = plt.imread(f, format='jpeg')
  # plt.imshow(image)
  inputs = feature_extractor(images=image, return_tensor="pt")
  pixel_values = inputs["pixel_values"]
  pixel_values = np.array(pixel_values)
  pixel_values = torch.tensor(pixel_values)
  outputs = model(pixel_values)
  logits = outputs.logits
  predicted_class_idx = logits.argmax(-1).item()
  sorted_preds = torch.argsort(logits, descending=True)[0]
  top_predictions = [sorted_preds[i].item() for i in range(50)] # 50 is semi-arbitrary based on our findings from testing pics
  top_values = [logits[0][pred].item() for pred in top_predictions]
  # print('label predictions', top_predictions)
  # print('values of predictions', top_values)
  found_cat_label = -1
  found_bad_label = -1
  bad_labels_found = []
  cat_score = 0
  for i, pred in enumerate(top_predictions):
    predicted_class = model.config.id2label[pred]
    # print(predicted_class)
    if pred in cat_labels:
      if found_cat_label == -1:
        found_cat_label = i
      cat_score += top_values[i]
    if pred in bad_labels:
      if found_bad_label == -1:
        found_bad_label = i
      bad_labels_found.append(pred)
      bad_labels_found.append(bad_labels[pred])
      cat_score -= top_values[i]
  # print(' ')
  print('    found cat label:', found_cat_label)
  print('    found bad label:', found_bad_label, bad_labels_found)
  would_pass = found_cat_label >= 0 and found_bad_label < 0
  # print('AI cat score: ', cat_score)
  # print('    passed cat test:', would_pass)
  return would_pass

In [16]:
# on colab you can set these userdata properties by clicking the key on the left bar, creating a secret, and giving the colab "notebook access"
BSKY_USERNAME = userdata.get('bsky_username')
BSKY_PASSWORD = userdata.get('bsky_password')

client = Client()
client.login(BSKY_USERNAME, BSKY_PASSWORD)

ProfileViewDetailed(did='did:plc:ktkc7jfakxzjpooj52ffc6ra', handle='tyrowo.bsky.social', associated=ProfileAssociated(chat=None, feedgens=0, labeler=False, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated'), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:ktkc7jfakxzjpooj52ffc6ra/bafkreiepqgg5tlozvv4bw5ficwwixflfprionhnli26hngc3hjn6ygpamu@jpeg', banner='https://cdn.bsky.app/img/banner/plain/did:plc:ktkc7jfakxzjpooj52ffc6ra/bafkreierasgybimbz3pfobqfmay5ufxm7jlqnjeiumdx7dyof6mcss7a4m@jpeg', created_at='2023-08-28T14:18:41.780Z', description='Self-taught professional software developer, exF2P MTG Arena Challenger, ex wr holder speedrunning HotD2100%, ex streamer, ex melee player. \nGrilling, coding, magic, and posting pictures of my cat. \nNO POLITICS\nhe/him', display_name='Tyro, ft Ricky', followers_count=818, follows_count=2402, indexed_at='2025-01-19T01:42:29.013Z', joined_via_starter_pack=None, labels=[], pinned_post=Main(cid='bafyreihszjgxkh5fmu5je5eldv

In [17]:
def like_post_and_add_user(post):
  user_did = post.author.did
  followed_user = client.follow(user_did).uri
  post_cid = post.cid
  post_uri = post.uri
  liked_post = client.like(uri=post_uri, cid=post_cid).uri
  print(f'      ✓✓✓ Successfully liked post and followed user: {post.author.handle}')
  return followed_user
  # TODO - add this friend and the followed-user uri to the database

In [26]:
# input variables - x is target follows y is num of posts we want to look through, whichever end criteria we reach first
EMBEDDED_PIC = 'app.bsky.embed.images#view'
EMBEDDED_VID = 'app.bsky.embed.video#view'
FEED_CATURDAY = 'at://did:plc:pmyqirafcp3jqdhrl7crpq7t/app.bsky.feed.generator/aaad4sb7tyvjw'
FEED_SIAMESE = 'at://did:plc:jv3qdc5vxujp6taaa7nte35i/app.bsky.feed.generator/aaac6wmikqyhq'
FEED_CATPICS = 'at://did:plc:q6gjnaw2blty4crticxkmujt/app.bsky.feed.generator/cv:cat'
FEED_CATS = 'at://did:plc:jfhpnnst6flqway4eaeqzj2a/app.bsky.feed.generator/cats'
FEED_NAME = {FEED_CATURDAY: "'Caturday'", FEED_SIAMESE: "'Siamese Cats'", FEED_CATPICS: "'Cat Pics'", FEED_CATS: "'Cats!'"}
URL_BEGIN = 'https://bsky.app/profile/'
URL_POST = '/post/'

# query creation constants
SCHEMA = 'bsky'
TABLE_FOLLOWS = 'follows'
COL_USERID = 'user_id'
COL_USERHANDLE = 'user_handle'
COL_FOLLOWURI = 'follow_uri'
COL_FOLLOWDATE = 'follow_date'
COL_FOLLOWSYOU = 'follows_you'
DATETIME_NOW = 'NOW()'
END_LINE = ','
END_QUERY = ';'


def createPostUrl(feed_post):
  url_handle = feed_post.post.author.handle
  url_ending_index = feed_post.post.uri.find('.feed.post/') + 11
  url_ending = feed_post.post.uri[url_ending_index : ]
  return URL_BEGIN + url_handle + URL_POST + url_ending

# TODO - The way I have this written the function probably breaks if we reach the end of the feed, so I need to write some kind of case to handle that. The feeds I'm using there shouldn't be an end to reach.
# But technically if you set the number of posts to check to like a billion and number of new followers to follow to like a billion it could reach the end and break.
# so please use responsibly

def follow_more_users(post_count, follows_count, feed):
  posts_to_check = post_count
  next_page = ''
  new_follow_count = 0
  page_count = 0
  query_created_of_users_added = []
  query_created_of_users_added.append(f'INSERT INTO "{SCHEMA}"."{TABLE_FOLLOWS}" ({COL_USERID}, {COL_USERHANDLE}, {COL_FOLLOWURI}, {COL_FOLLOWDATE}, {COL_FOLLOWSYOU}) ')
  query_created_of_users_added.append(f'VALUES ')
  set_of_user_rows_added = set()
  while posts_to_check > 0:
    print(f'[checking page {page_count} of feed {FEED_NAME[feed]}, {posts_to_check} posts left to check, and have found {new_follow_count} new users to follow]')
    page_count += 1
    limit = min(posts_to_check, 100)
    posts_to_check -= limit
    data = client.app.bsky.feed.get_feed({
        'feed': 'at://did:plc:jfhpnnst6flqway4eaeqzj2a/app.bsky.feed.generator/cats',
        'limit': limit,
        'cursor': next_page
    }, headers={})
    next_page = data.cursor

    for i, f in enumerate(data.feed):
      you_follow_them = f.post.author.viewer.following
      you_are_followed_by = f.post.author.viewer.followed_by

      if False: # TODO IF POST ALREADY SEEN
        print(f'{i} - skipped. Post {""} has already been viewed.') # add post id here idk
        continue
      else:
        # TODO - add post did to set to mark as seen
        if you_follow_them and you_are_followed_by:
          print(f'{i} 💕 user: {f.post.author.handle} is a mutual follower. Liking this post. {createPostUrl(f)}')
          liked_post = client.like(uri=f.post.uri, cid=f.post.cid).uri
        elif you_follow_them or you_are_followed_by:
          print(f'{i} ✗ user: {f.post.author.handle} {"already follows you." if you_are_followed_by else ""}{"is already being followed." if you_follow_them else ""}')
        ## TODO - add elif check here for if they are in the database as someone who was once our friend? not necessary, could just immediately cycle them back into the friend rotation, but do this if you don't want to readd people
        elif not f.post.embed or f.post.embed.py_type != EMBEDDED_PIC:
          if f.post.embed.py_type == EMBEDDED_VID:
            print(f'{i} ✗ 🎥 video post: {createPostUrl(f)}')
          else:
            print(f'{i} ✗ no pic for post {i}')
        else:
          print(i, '✓', f.post.embed.images[0].fullsize)
          print(f'    post: {createPostUrl(f)}')
          is_cat = test_bsky_image(f.post.embed.images[0].fullsize)
          if is_cat:
            print(f'    ✓✓ successfully found cat pic at post {i}.')
            new_follow_count += 1
            did = f.post.author.did
            handle = f.post.author.handle
            follow_uri = like_post_and_add_user(f.post)
            date_added = DATETIME_NOW
            follows_you = False
            if len(query_created_of_users_added) > 2:
              # first one don't add a comma but add one after every one after that
              query_created_of_users_added.append(END_LINE)
            query_created_of_users_added.append(f"    ('{did}', '{handle}', '{follow_uri}', {date_added}, '{follows_you}') ")
            if new_follow_count == follows_count:
              print(f'Successfully followed {new_follow_count} new users!')
              query_created_of_users_added.append(f'RETURNING {COL_USERHANDLE}{END_QUERY}')
              return query_created_of_users_added
          else:
            print(f'    ✓✗ post {i} was not a cat pic')

  print(f'ran out of posts to check for new followers. followed {new_follow_count} new users.')
  query_created_of_users_added.append(f'RETURNING {COL_USERHANDLE}{END_QUERY}')
  return query_created_of_users_added

    # print(data.feed[0].post.embed.images[0].fullsize)

In [27]:
CATURDAY_DOW = 'Saturday'
USER_TIMEZONE = "US/Eastern" # you should fill this in with your own timezone here
cur_timestamp = datetime.datetime.now(zoneinfo.ZoneInfo(USER_TIMEZONE))

dow = cur_timestamp.strftime("%A")
is_caturday = dow == CATURDAY_DOW
query_caturday = []

if is_caturday:
  print("IT'S CATURDAY! Checking the Caturday feed for new followers.")
  query_caturday = follow_more_users(500, 100, FEED_CATURDAY)
else:
  print("Sorry, today's not Caturday.")

Sorry, today's not Caturday.


In [28]:
for line in query_caturday:
  print(line)

In [29]:
query_siamese = follow_more_users(500, 100, FEED_SIAMESE)

[checking page 0 of feed 'Siamese Cats', 500 posts left to check, and have found 0 new users to follow]
0 ✗ user: crestfallenchimera.bsky.social is already being followed.
1 ✗ user: anticyclone.bsky.social is already being followed.
2 ✗ user: adjcent2u.bsky.social is already being followed.
3 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:cqwyznxwm3gfum332pmxadjp/bafkreicano5hrjl6kbogvqhlmk3hboyitow4v57k56fr5td3ltpdxw37h4@jpeg
    post: https://bsky.app/profile/westalguy.bsky.social/post/3lg5j5nk3wc2i
    found cat label: -1
    found bad label: -1 []
    ✓✗ post 3 was not a cat pic
4 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:wxv4sfwjm7s2lq4lcclrb57q/bafkreifcasiya6r5gf7tfbg3vf5qp2oh64rb3rg6ub7xxscyifvg2rryey@jpeg
    post: https://bsky.app/profile/macabrett.bsky.social/post/3lg5j5megts2e
    found cat label: 1
    found bad label: -1 []
    ✓✓ successfully found cat pic at post 4.
      ✓✓✓ Successfully liked post and followed user: macabrett.bsky.social
None
5 ✓ 

In [30]:
for line in query_siamese:
  print(line)

INSERT INTO "bsky"."follows" (user_id, user_handle, follow_uri, follow_date, follows_you) 
VALUES 
    ('did:plc:wxv4sfwjm7s2lq4lcclrb57q', 'macabrett.bsky.social', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lg5juqmr5a26', NOW(), 'False') 
,
    ('did:plc:akryjwryyqj55wbrvv6nkyys', 'karlaliliana.bsky.social', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lg5jutpqki23', NOW(), 'False') 
,
    ('did:plc:exouihy2r5erqfysnjxpiaho', 'big-sir.bsky.social', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lg5juvuea722', NOW(), 'False') 
,
    ('did:plc:po3amig2gjr5elm4zuztdedx', 'joematango.bsky.social', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lg5juxzelx2m', NOW(), 'False') 
,
    ('did:plc:mrp4ku6en2fzfykawov5sada', 'motleybeast.bsky.social', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lg5juz2b4i2c', NOW(), 'False') 
,
    ('did:plc:aacizd3tmy26xo4ax5njl2kd', 'trashm.ink', 'at://did:plc:ktkc7jfakxzjpooj52

In [31]:
query_cats = follow_more_users(500, 200, FEED_CATS)

[checking page 0 of feed 'Cats!', 500 posts left to check, and have found 0 new users to follow]
0 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:orczy2tyzs4bme2sskttu2sc/bafkreiflxrvgbagpofs24s7zntwkoykmcrycnxlx724wxe6c6ifrvckx2e@jpeg
    post: https://bsky.app/profile/niilvene.bsky.social/post/3lg5jq7rda227
    found cat label: 0
    found bad label: -1 []
    ✓✓ successfully found cat pic at post 0.
      ✓✓✓ Successfully liked post and followed user: niilvene.bsky.social
None
1 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:7crxkzpmdd453ojocu4zjyoy/bafkreidizn6s2t5necqcea3c22f6nnqfrqpkkx76mfo4c3pjywgxkzatne@jpeg
    post: https://bsky.app/profile/patcat13.bsky.social/post/3lg5jpr3c7k2o
    found cat label: -1
    found bad label: -1 []
    ✓✗ post 1 was not a cat pic
2 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:m4sao25ogqf3thrlwxzo3duj/bafkreicefjvrbxh36524stm2ui2tsag2iopittffrttpgqtd33unarwlja@jpeg
    post: https://bsky.app/profile/abyssalgaze.bsky.soc

In [46]:
set_lines = set()
for line in query_cats[2:-1]:
  # print(line)
  set_lines.add(line)
# print(set_lines)
print(query_cats[0])
print(query_cats[1])
set_lines.remove(',')
for i, line in enumerate(set_lines):
  if i != 0:
    print(',')
  print(line)
print(query_cats[-1])

# for line in query_cats:
#   print(line)

INSERT INTO "bsky"."follows" (user_id, user_handle, follow_uri, follow_date, follows_you) 
VALUES 
    ('did:plc:mgc6ax2d2n5fi75lpmkbf2wo', 'daniandi4.bsky.social', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lg5ka3krr72h', NOW(), 'False') 
,
    ('did:plc:fi2vmc5kfbrlsvglhvpf5dcy', 'waynereewrites.com', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lg5k7gtszk22', NOW(), 'False') 
,
    ('did:plc:3uzsnlwen2oa7q7bgbbki3tu', 'suddenlymicah.bsky.social', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lg5keegkph22', NOW(), 'False') 
,
    ('did:plc:fuk67atin53p5yqetynmrjts', 'lavender-lemon.bsky.social', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lg5kbck6ji2c', NOW(), 'False') 
,
    ('did:plc:maydb63u4rzkjnoe2r3fwklj', 'chrispco.bsky.social', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lg5kbemfod2a', NOW(), 'False') 
,
    ('did:plc:qsukxwymdqdp6aefykwtri66', 'docellen.bsky.social', 'at://did:plc:ktkc7jf

In [33]:
query_catpics = follow_more_users(200, 50, FEED_CATPICS)

[checking page 0 of feed 'Cat Pics', 200 posts left to check, and have found 0 new users to follow]
0 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:azz2h6zwp7fnl5lwvq56y3gs/bafkreigxjzwtlzhd7jwomvfkirpebtqe3lfa3kbhby5tbrs7ecilu2g7vq@jpeg
    post: https://bsky.app/profile/altlifeuk.bsky.social/post/3lg5jzgczcc2p
    found cat label: 0
    found bad label: -1 []
    ✓✓ successfully found cat pic at post 0.
      ✓✓✓ Successfully liked post and followed user: altlifeuk.bsky.social
None
1 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:2a7d4don4xplfjvuh2fvd5od/bafkreify7lbmb3k4wvjbo7nonv74m743riqipqqkg6vk4x4l2avccxsesu@jpeg
    post: https://bsky.app/profile/brambleborn.bsky.social/post/3lg5jyp3bik2q
    found cat label: 0
    found bad label: -1 []
    ✓✓ successfully found cat pic at post 1.
      ✓✓✓ Successfully liked post and followed user: brambleborn.bsky.social
None
2 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:a7egwkafbtaaywwxthdilopw/bafkreial5co6mfadp

In [34]:
for line in query_catpics:
  print(line)

INSERT INTO "bsky"."follows" (user_id, user_handle, follow_uri, follow_date, follows_you) 
VALUES 
    ('did:plc:azz2h6zwp7fnl5lwvq56y3gs', 'altlifeuk.bsky.social', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lg5kfgigta2c', NOW(), 'False') 
,
    ('did:plc:2a7d4don4xplfjvuh2fvd5od', 'brambleborn.bsky.social', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lg5kfhx7qx22', NOW(), 'False') 
,
    ('did:plc:a7egwkafbtaaywwxthdilopw', 'olirats.bsky.social', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lg5kfjg3vi2c', NOW(), 'False') 
,
    ('did:plc:u4kx2yaum5fyxdapf5dacfbd', 'itstayyo.bsky.social', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lg5kfmhgtf2o', NOW(), 'False') 
,
    ('did:plc:kykkxnegmg4ki4zosfzyfakk', 'aussieamigos.bsky.social', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lg5kfngl6m2c', NOW(), 'False') 
,
    ('did:plc:eurywtxer45qp6vn7ltkkvxs', 'therealrnann.bsky.social', 'at://did:plc:ktkc7j